In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install transformers datasets peft accelerate trl sentencepiece



In [ ]:
from datasets import load_dataset

# Load the Alpaca-cleaned dataset
dataset = load_dataset("yahma/alpaca-cleaned")
print(dataset["train"][0])  # Check the first example in the training set


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


alpaca_data_cleaned.json:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

{'output': '1. Eat a balanced and nutritious diet: Make sure your meals are inclusive of a variety of fruits and vegetables, lean protein, whole grains, and healthy fats. This helps to provide your body with the essential nutrients to function at its best and can help prevent chronic diseases.\n\n2. Engage in regular physical activity: Exercise is crucial for maintaining strong bones, muscles, and cardiovascular health. Aim for at least 150 minutes of moderate aerobic exercise or 75 minutes of vigorous exercise each week.\n\n3. Get enough sleep: Getting enough quality sleep is crucial for physical and mental well-being. It helps to regulate mood, improve cognitive function, and supports healthy growth and immune function. Aim for 7-9 hours of sleep each night.', 'input': '', 'instruction': 'Give three tips for staying healthy.'}


In [ ]:
!apt-get install git-lfs


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

print("Model and tokenizer loaded successfully!")


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Model and tokenizer loaded successfully!


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
from transformers import Trainer, DataCollatorForLanguageModeling
from datasets import load_dataset

# Load model and tokenizer
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
dataset = load_dataset("yahma/alpaca-cleaned")["train"]
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def tokenize(example):
    return tokenizer(f"### Instruction:\n{example['instruction']}\n### Input:\n{example['input']}\n### Response:\n{example['output']}", padding="max_length", truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize)

# Load the model
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set up LoRA configuration
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)
model = get_peft_model(model, config)

# Training arguments (disable W&B logging)
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=10,
    max_steps=500,  # Limit total steps to 500 for quick testing
    learning_rate=2e-4,
    fp16=True,
    logging_steps=20,
    save_steps=250,
    save_total_limit=2,
    logging_dir="./logs",
    report_to="none",  # Disable W&B for now to simplify
)


# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

# Fine-tune the model
trainer.train()


<ipython-input-4-d2383ca73f22>:50: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
20,1.483400
40,1.141500
60,1.165900
80,1.166500
100,1.208100
120,1.148500
140,1.160900
160,1.167400
180,1.103200
200,1.142400


TrainOutput(global_step=500, training_loss=1.150195556640625, metrics={'train_runtime': 768.7246, 'train_samples_per_second': 5.203, 'train_steps_per_second': 0.65, 'total_flos': 1.2725929377792e+16, 'train_loss': 1.150195556640625, 'epoch': 0.07727975270479134})

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Path to your Hugging Face model repository or local files
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Or the local path to the model
tokenizer_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Or the local path to the tokenizer

# Load tokenizer and model from Hugging Face (or use local paths if the model is saved locally)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32)

# Prepare for inference
model.eval()
if torch.cuda.is_available():
    model.to("cuda")

prompt = "### Instruction:\nExplain the concept of quantum computing in simple terms.\n### Input:\n\n### Response:\n"
inputs = tokenizer(prompt, return_tensors="pt")

# Move inputs to the GPU if available
if torch.cuda.is_available():
    inputs = {k: v.to("cuda") for k, v in inputs.items()}

# Generate output
outputs = model.generate(
    **inputs,
    max_new_tokens=150,
    do_sample=True,
    temperature=0.7,
    top_p=0.9
)

# Decode and print
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### Instruction:
Explain the concept of quantum computing in simple terms.
### Input:

### Response:
Quantum computing is a branch of computer science that operates on the quantum mechanical principles of matter and energy. It is a powerful and complex field that involves the use of quantum bits, or qubits, to perform calculations that are beyond the capabilities of classical computers. Quantum computing has the potential to solve complex problems that are currently infeasible for classical computers, such as protein folding, drug design, and cryptography. However, it is still in the early stages of development and requires significant investment and research to achieve commercial viability.


In [ ]:
!apt-get install git


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
# Replace with your own name and email associated with GitHub
!git config --global user.name "Anushka1702"
!git config --global user.email "kishoranushka1702.com"


In [ ]:
!git clone https://github.com/Anushka1702/tinyllama-finetune.git


Cloning into 'tinyllama-finetune'...


In [ ]:
!cp /content/drive/MyDrive/tinyllama-finetune/finetune.py /content/tinyllama-finetune/
!cp /content/drive/MyDrive/tinyllama-finetune/inference.py /content/tinyllama-finetune/



cp: cannot stat '/content/drive/MyDrive/tinyllama-finetune/finetune.py': No such file or directory
cp: cannot stat '/content/drive/MyDrive/tinyllama-finetune/inference.py': No such file or directory
